In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ase #package called Atomic Simulation Environment which allows us to construct crystals easily
from ase.spacegroup import crystal
from ase.visualize import view 
from ase.visualize.plot import plot_atoms
from tqdm import tqdm
from ase.io import read
from ase.io import write

In [ ]:
'''

This script is designed to generate a large scale crystal meant to import into a LAMMPS script using the ASE package. 
The input involves searching up a .cif file corresponding to the unit cell structure of that particularly
desired crystal. This unit cell can be multiplied many times into a much larger supercell structure. 

@author: kmream
'''

In [19]:
# Read CIF file -> these must be downloaded and located in the same repository 
#atoms = read('1011176.cif') #quartz
atoms = read('9011998.cif') #silicon 

# Replicate the unit cell
atoms *= (5, 5, 5)

view(atoms) 

<Popen: returncode: None args: ['/opt/anaconda3/bin/python', '-m', 'ase', 'g...>

In [12]:
write("silicon_supercell.lmp", atoms, format="lammps-data", specorder=['Si'])

with open('silicon_supercell.lmp', 'w') as f:
    f.write("ID Type x y z\n")
    for i, atom in enumerate(atoms, start=1):
        atom_type = atom.number  # or use a mapping from symbol if needed
        x, y, z = atom.position
        f.write(f"{i} {atom_type} {x:.6f} {y:.6f} {z:.6f}\n")  

In [13]:
data = pd.read_csv('silicon_supercell.txt', sep = " ")
data['ID'] = data.index + 14
data.loc[data['Type'] == 14, 'Type'] = 1
#data.loc[data['Type'] == 8, 'Type'] = 2
charge_map = {1: 0.0}#, 2: -0.5}
data['charge'] = data['Type'].map(charge_map)
charge_col = data.pop('charge')
data.insert(2, 'charge', charge_col)
data

,ID,Type,charge,x,y,z
0,14,1,0.0,0.000000,0.000000,0.000000
1,15,1,0.0,0.000000,2.715470,2.715470
2,16,1,0.0,2.715470,0.000000,2.715470
3,17,1,0.0,2.715470,2.715470,0.000000
4,18,1,0.0,4.073206,4.073206,1.357735
...,...,...,...,...,...,...
999995,1000009,1,0.0,268.831579,268.831579,266.116109
999996,1000010,1,0.0,270.189315,270.189315,267.473844
999997,1000011,1,0.0,270.189315,267.473844,270.189315
999998,1000012,1,0.0,267.473844,270.189315,270.189315


In [16]:
def write_lammps_data(df, filename):
    with open(filename, 'w') as f:
        f.write("#LAMMPS data file via pandas\n\n")
        f.write(f"{len(df)} atoms\n")
        f.write(f"{df['Type'].nunique() + 1} atom types\n\n")

        # Assuming orthogonal box and bounds can be inferred from coordinates
        xlo, xhi = df['x'].min(), df['x'].max()
        ylo, yhi = df['y'].min(), df['y'].max()
        zlo, zhi = df['z'].min(), df['z'].max()

        # Add some padding
        pad = 3.0
        f.write(f"{xlo - pad:.6f} {xhi + pad:.6f} xlo xhi\n")
        #f.write(f"{ylo - pad:.6f} {yhi + pad:.6f} ylo yhi\n")
        f.write(f"-100 400 ylo yhi\n")
        f.write(f"{zlo - pad:.6f} {zhi + pad:.6f} zlo zhi\n\n")
        
        f.write("Masses\n\n")
        f.write(f" 1 28.08500000 # Si\n")
        #f.write(f" 2 15.99900000 # O\n")
        f.write(f" 2 196.9665500 # Au\n\n")

        f.write("Atoms\n\n")
        for _, row in df.iterrows():
            f.write(f"{int(row['ID'])} {int(row['Type'])} {int(row['charge'])} {row['x']} {row['y']} {row['z']}\n")

# Usage
#write_lammps_data(data, 'quartz_supercell2.lmp')

In [17]:
write_lammps_data(data, 'silicon_supercell.lmp')